<a href="https://colab.research.google.com/github/RSid8/PCAPLAB/blob/main/CudaLab6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

%load_ext nvcc_plugin

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-_gaorscz
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-_gaorscz
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp37-none-any.whl size=4307 sha256=96b27989032867f07a90dad2f12f9dd9f32d3f057818526625300d2ab7c47277
  Stored in directory: /tmp/pip-ephem-wheel-cache-0djzdv5l/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [4]:
%%cu
#include<stdio.h>
#include<stdlib.h>

__global__ void addMatrixKernel_1a(int * A, int *B , int *C, int M)
{
    int idx =threadIdx.x;
    for(int j=0;j<M;j++)
    {
        C[idx*M+j]=A[idx*M+j]+B[idx*M+j];
    }
}

__global__ void addMatrixKernel_1b(int * A, int *B , int *C, int N)
{
    int idx =threadIdx.x;
    for(int i=0;i<N;i++)
    {
        C[i*N+idx]=A[i*N+idx]+B[i*N+idx];
    }
}

__global__ void addMatrixKernel_1c(int * A, int *B , int *C, int M)
{
    int idx =blockIdx.x*blockDim.x+threadIdx.x;
    C[idx]=A[idx]+B[idx];
}

const int N=3;
const int M=3;

void printMatrix(int A[M][N])
{
    for(int i=0;i<N;i++)
    {
        for(int j=0;j<M;j++)
        {
            printf("%d ",A[i][j]);
        }
        printf("\n");
    }
}



int main()
{
    
 
    int A[N][M];
    int B[N][M];
    int C[N][M];
 
    printf("Matrix A:\n");
    for(int i=0;i<N;i++)
    {
        for(int j=0;j<M;j++)
        {
            A[i][j]=(rand()%30);
            printf("%d ",A[i][j]);
        }
        printf("\n");
    }
 
    printf("\n\n");
    printf("Matrix B:\n");
 
    for(int i=0;i<N;i++)
    {
        for(int j=0;j<M;j++)
        {
            B[i][j]=(rand()%25);
            printf("%d ",B[i][j]);
        }
        printf("\n");
    }
    

    int size=sizeof(int)*N*M;
    int *d_a,*d_b,*d_c;
 
    cudaMalloc((void**)&d_a,size);
    cudaMalloc((void**)&d_b,size);
    cudaMalloc((void**)&d_c,size);

    cudaMemcpy(d_a,A,size,cudaMemcpyHostToDevice);
    cudaMemcpy(d_b,B,size,cudaMemcpyHostToDevice);

    addMatrixKernel_1a<<<1,N>>>(d_a,d_b,d_c,M);
 
    cudaMemcpy(C,d_c,size,cudaMemcpyDeviceToHost);

    printf("\n\n");
    printf("Resultant matrix- 1a :\n");
 
    printMatrix(C);


    addMatrixKernel_1b<<<1,N>>>(d_a,d_b,d_c,N);
 
    cudaMemcpy(C,d_c,size,cudaMemcpyDeviceToHost);

    printf("\n\n");
    printf("Resultant matrix- 1b :\n");
 
    printMatrix(C);


    addMatrixKernel_1c<<<N,M>>>(d_a,d_b,d_c,M);
 
    cudaMemcpy(C,d_c,size,cudaMemcpyDeviceToHost);

    printf("\n\n");
    printf("Resultant matrix- 1c :\n");

    printMatrix(C);
    
    return 0;

}


Matrix A:
13 16 27 
25 23 25 
16 12 9 


Matrix B:
21 12 2 
15 9 13 
1 15 1 


Resultant matrix- 1a :
34 28 29 
40 32 38 
17 27 10 


Resultant matrix- 1b :
34 28 29 
40 32 38 
17 27 10 


Resultant matrix- 1c :
34 28 29 
40 32 38 
17 27 10 



In [5]:
%%cu
#include<stdio.h>
#include<stdlib.h>

__global__ void multiplyMatrix_row(int * A, int *B , int *C,int wa, int wb)
{
    int idx =threadIdx.x;

        for(int j=0;j<wb;j++)
       {
           int sum=0;
           for(int k=0;k<wa;k++)
           {
               sum=sum+A[idx*wa+k]*B[k*wb+j];
           }
           C[idx*wb+j]=sum;
       }
}

__global__ void multiplyMatrix_col(int * A, int *B , int *C,int wa, int wb,int ha)
{
    int idx =threadIdx.x;
    for(int i=0;i<ha;i++)
    {
           int sum=0;
           for(int k=0;k<wa;k++)
           {
               sum=sum+A[i*wa+k]*B[k*wb+idx];
           }
           C[i*wb+idx]=sum;
    }
}

__global__ void multiplyMatrix_element(int * A, int *B , int *C,int wa, int wb,int ha)
{
    int row =blockIdx.x;
    int col=threadIdx.x;
    int sum=0;
 
    for(int k=0;k<wa;k++)
    {
        sum=sum+A[row*wa+k]*B[k*wb+col];
    }
    C[row*wb+col]=sum;
}


const int ha=3;
const int wa=3;
 
const int hb=3;
const int wb=3;
 
void printMatrix(int A[ha][wb])
{
    for(int i=0;i<ha;i++)
    {
        for(int j=0;j<wb;j++)
        {
            printf("%d ",A[i][j]);
        }
        printf("\n");
    }
}

int main()
{
    
    int A[ha][wa];
    int B[hb][wb];
    int C[ha][wb];
 
    if(wa!=hb)
      {
          printf("Dimenssions are not correct! ");
          return 0;
      }
 
    for(int i=0;i<ha;i++)
    {
        for(int j=0;j<wb;j++)
        {
            A[i][j]=(rand()%30);
        }
    }
    for(int i=0;i<hb;i++)
    {
        for(int j=0;j<wb;j++)
        {
            B[i][j]=(rand()%25);
        }
    }
    printf("Matrix A:\n");
    for(int i=0;i<ha;i++)
    {
        for(int j=0;j<wa;j++)
        {
            printf("%d ",A[i][j]);
        }
        printf("\n");
    }
 
    printf("\n\n");
    printf("Matrix B:\n");
    for(int i=0;i<hb;i++)
    {
        for(int j=0;j<wb;j++)
        {
            printf("%d ",B[i][j]);
        }
        printf("\n");
    }

    int sizeA=sizeof(int)*ha*wa;
    int sizeB=sizeof(int)*hb*wb;
    int sizeC=sizeof(int)*ha*wb;
 
 
    int *d_a,*d_b,*d_c;
 
    cudaMalloc((void**)&d_a,sizeA);
    cudaMalloc((void**)&d_b,sizeB);
    cudaMalloc((void**)&d_c,sizeC);
 

    cudaMemcpy(d_a,A,sizeA,cudaMemcpyHostToDevice);
    cudaMemcpy(d_b,B,sizeB,cudaMemcpyHostToDevice);

    multiplyMatrix_row<<<1,ha>>>(d_a,d_b,d_c,wa,wb);
 
    cudaMemcpy(C,d_c,sizeC,cudaMemcpyDeviceToHost);

    printf("\n\n");
    printf("Resultant matrix 2a:\n");
    printMatrix(C);
 
    multiplyMatrix_col<<<1,wb>>>(d_a,d_b,d_c,wa,wb,ha);
 
    cudaMemcpy(C,d_c,sizeC,cudaMemcpyDeviceToHost);

    printf("\n\n");
    printf("Resultant matrix 2b:\n");
    printMatrix(C);
 
    multiplyMatrix_element<<<ha,wb>>>(d_a,d_b,d_c,wa,wb,ha);
 
    cudaMemcpy(C,d_c,sizeC,cudaMemcpyDeviceToHost);

    printf("\n\n");
    printf("Resultant matrix 2c:\n");
    printMatrix(C);


    
    return 0;

}

Matrix A:
13 16 27 
25 23 25 
16 12 9 


Matrix B:
21 12 2 
15 9 13 
1 15 1 


Resultant matrix 2a:
540 705 261 
895 882 374 
525 435 197 


Resultant matrix 2b:
540 705 261 
895 882 374 
525 435 197 


Resultant matrix 2c:
540 705 261 
895 882 374 
525 435 197 

